In [5]:
import os
import re
import json
import random
import sys
import asyncio
import pickle
import datetime
import gurobipy as gp
sys.path.append('../')

from openai import OpenAI, AsyncClient
from json import JSONDecodeError
from enum import Enum
from tqdm.auto import tqdm
from utils import *
from pydantic import BaseModel
from colorama import Fore, Style
from llama_index.program.openai import OpenAIPydanticProgram
from llama_index.llms.openai import OpenAI

In [6]:
cfg = json.load(open('../configs./configs.json', 'r'))
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
asyncclient = AsyncClient(api_key=os.environ["OPENAI_API_KEY"])

dt = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')

In [7]:
DATA_DIR = '../data'
DATASET_NAME = 'LPWP' 
OUTPUT_DIR = '../output'  

nl4opt_data = read_txt_file(os.path.join(DATA_DIR, DATASET_NAME, 'lpwp.txt'))
questions, answers = get_nl4opt_qas(nl4opt_data)
assert len(questions) == len(answers)

qa_pairs = list(zip(questions, answers))
# demo_samples, test_samples = get_demo_and_test_samples(qa_pairs)

questions = [q for q, _ in qa_pairs]
answers = [a for _, a in qa_pairs]

2024-09-19 14:35:40.103 | DEBUG    | utils:read_txt_file:15 - Reading file: ../data\LPWP\lpwp.txt
2024-09-19 14:35:40.105 | DEBUG    | utils:read_txt_file:17 - File read successfully: ../data\LPWP\lpwp.txt
2024-09-19 14:35:40.133 | INFO     | utils:get_nl4opt_qas:36 - Number of questions: 288
2024-09-19 14:35:40.134 | INFO     | utils:get_nl4opt_qas:37 - Number of answers: 288


In [8]:
# class VarType(Enum):
#     INTEGER = "INTEGER"
#     CONTINUOUS = "CONTINUOUS"

class CodeScheme(BaseModel):
    """The variable type in the problem and the code string"""
    
    # var_type: VarType
    code: str     

In [9]:
sys_prompt = """You are an expert in optimization problems and domain specific language generation. 
Your task is to convert the textual optimization text into lines of code.
You should also analyze whether the variable in the optimization problem should be INTEGER or CONTINUOUS.
DO NOT ADD ANY COMMENTS OR EXPLANATION TO THE CODE. JUST OUTPUT THE CODE.
Here are some examples that you should refer to:\n"""

example = """
QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
VAR_TYPE:
INTEGER
CODE:
x = m.addVar(name="Oil Max", vtype=gp.GRB.INTEGER)
y = m.addVar(name="Oil Max Pro", vtype=gp.GRB.INTEGER)
m.setObjective(10 * x + 15 * y, gp.GRB.MAXIMIZE)
m.addConstr(46 * x + 13 * y <= 1345)
m.addConstr(43 * x + 4 * y <= 346)
m.addConstr(56 * x + 45 * y <= 1643)

QUESTION:
Ben is growing apples and pears on his orchard. He has 50 acres available on which he must grow a minimum of 5 acres of apples and a minimum of 10 acres of pears to meet demands. The profit per apple is $2 and the profit per pear is $4. He prefers to grow more pears than apples but limitations in his workforce allow him to grow at most twice the amount of pears as apples. How many of each fruit should Ben grow in order to maximize his profit? What is that profit?
VAR_TYPE:
INTEGER
CODE:
x = m.addVar(name="apples", vtype=gp.GRB.INTEGER)
y = m.addVar(name="pears", vtype=gp.GRB.INTEGER)
m.setObjective(2 * x + 4 * y, gp.GRB.MAXIMIZE)
m.addConstr(x + y <= 50)
m.addConstr(x >= 5)
m.addConstr(y >= 10)
m.addConstr(y <= 2 * x)
"""

prompt_template_str = sys_prompt + example + "\nPlease finish the task think step by step.\nQUESTION:{q}"
print(prompt_template_str)

You are an expert in optimization problems and domain specific language generation. 
Your task is to convert the textual optimization text into lines of code.
You should also analyze whether the variable in the optimization problem should be INTEGER or CONTINUOUS.
DO NOT ADD ANY COMMENTS OR EXPLANATION TO THE CODE. JUST OUTPUT THE CODE.
Here are some examples that you should refer to:

QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
VAR_

In [10]:
program = OpenAIPydanticProgram.from_defaults(
    output_cls=CodeScheme, 
    llm=OpenAI("gpt-3.5-turbo"),
    prompt_template_str=prompt_template_str.format(q=questions[2]), 
    verbose=True
)

In [11]:
code = program()

Function call: CodeScheme with args: {"code":"x = m.addVar(name=\"fertilizer A\", vtype=gp.GRB.CONTINUOUS)\ny = m.addVar(name=\"fertilizer B\", vtype=gp.GRB.CONTINUOUS)\nm.setObjective(5 * x + 9 * y, gp.GRB.MINIMIZE)\nm.addConstr(13 * x + 8 * y >= 220)\nm.addConstr(5 * x + 14 * y >= 160)\nm.addConstr(6 * x + 6 * y <= 350)"}


In [12]:
execute_code(complement_code(clean_code(code.code)))

125.49295774647888

In [13]:
# batch_size = 8
# lp_reasoning_list = []
# for idx in tqdm(range(0, len(questions), batch_size)):
#     batch = questions[idx:idx+batch_size]
    
#     tasks = [asyncclient.beta.chat.completions.parse(
#         model="gpt-4o-2024-08-06",
#         temperature=0,
#         response_format=Code,
#         messages=[
#             {"role": "system", "content": sys_prompt},
#             {"role": "user", "content": f"QUESTION: {q}"}
#         ]) for q in batch
#     ]

#     combined_responses = await asyncio.gather(*tasks)
#     lp_reasoning_list.extend([r.choices[0].message.parsed for r in combined_responses])

codes = []
for i in tqdm(range(len(questions))):
    program = OpenAIPydanticProgram.from_defaults(
        output_cls=CodeScheme, 
        llm=OpenAI("gpt-3.5-turbo"),
        prompt_template_str=prompt_template_str.format(q=questions[i]), 
        verbose=False
    )
    code = program()
    codes.append(code)

100%|██████████| 288/288 [10:53<00:00,  2.27s/it]


In [14]:
filename = 'e2e_codegen_gpt-3.5-turbo_lpwp_gurobi_' + dt + '.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'wb') as f:
    pickle.dump(codes, f)

In [15]:
codes = [codes[i].code for i in range(len(codes))]

In [16]:
prefix = """
import gurobipy as gp
env = gp.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()
m = gp.Model(env=env)
"""
                
suffix = """
m.optimize()
"""

def complement_code(code: str) -> float:
    return prefix + code + suffix

In [17]:
def clean_code(code: str) -> str:
    cleand_code = []
    for line in code.split('\n'):
        line = line.strip()
        if line.startswith('m.addConstr') and not re.findall(r'<=|>=', line):
            line = re.sub(r'<', r'<=', line)
            line = re.sub(r'>', r'>=', line)
        cleand_code.append(line)
    cleand_code = '\n'.join(cleand_code)
    cleand_code = cleand_code.replace(')m', ')\nm')
    return cleand_code

In [18]:
def execute_code(code: str) -> float:
    ex_locals = {}
    exec(code, None, ex_locals)
    
    try:
        return ex_locals["m"].objVal
    except Exception as e:
        # print(e)
        return np.inf

In [19]:
pred_answers = []
for i, code_str in enumerate(codes):
    try:
        cleaned_code = clean_code(code_str)
        code = complement_code(cleaned_code)
        ans = execute_code(code)
        loguru.logger.info(f"question {i} obtain answer")
        pred_answers.append(ans)
    except Exception as e:
        loguru.logger.error(f"Error for question {i}: {e}")
        pred_answers.append("Error")

2024-09-19 14:52:43.355 | ERROR    | __main__:<module>:10 - Error for question 0: name 'cost_sled_dogs' is not defined
2024-09-19 14:52:43.357 | INFO     | __main__:<module>:7 - question 1 obtain answer
2024-09-19 14:52:43.358 | INFO     | __main__:<module>:7 - question 2 obtain answer
2024-09-19 14:52:43.361 | INFO     | __main__:<module>:7 - question 3 obtain answer
2024-09-19 14:52:43.364 | INFO     | __main__:<module>:7 - question 4 obtain answer
2024-09-19 14:52:43.366 | INFO     | __main__:<module>:7 - question 5 obtain answer
2024-09-19 14:52:43.367 | INFO     | __main__:<module>:7 - question 6 obtain answer
2024-09-19 14:52:43.369 | INFO     | __main__:<module>:7 - question 7 obtain answer
2024-09-19 14:52:43.370 | INFO     | __main__:<module>:7 - question 8 obtain answer
2024-09-19 14:52:43.372 | INFO     | __main__:<module>:7 - question 9 obtain answer
2024-09-19 14:52:43.375 | INFO     | __main__:<module>:7 - question 10 obtain answer
2024-09-19 14:52:43.379 | INFO     | __m

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-17
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros
Model fingerprint: 0xfd346e33
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [5e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 8e+05]
Presolve removed 3 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.8400000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.840000000e+05


2024-09-19 14:52:43.394 | INFO     | __main__:<module>:7 - question 13 obtain answer
2024-09-19 14:52:43.396 | INFO     | __main__:<module>:7 - question 14 obtain answer
2024-09-19 14:52:43.398 | INFO     | __main__:<module>:7 - question 15 obtain answer
2024-09-19 14:52:43.400 | INFO     | __main__:<module>:7 - question 16 obtain answer
2024-09-19 14:52:43.402 | INFO     | __main__:<module>:7 - question 17 obtain answer
2024-09-19 14:52:43.404 | INFO     | __main__:<module>:7 - question 18 obtain answer
2024-09-19 14:52:43.406 | INFO     | __main__:<module>:7 - question 19 obtain answer
2024-09-19 14:52:43.408 | INFO     | __main__:<module>:7 - question 20 obtain answer
2024-09-19 14:52:43.412 | INFO     | __main__:<module>:7 - question 21 obtain answer
2024-09-19 14:52:43.414 | INFO     | __main__:<module>:7 - question 22 obtain answer
2024-09-19 14:52:43.419 | INFO     | __main__:<module>:7 - question 23 obtain answer
2024-09-19 14:52:43.421 | INFO     | __main__:<module>:7 - questi

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x567311ae
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [3e-01, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 8e+00]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.000000e+00   0.000000e+00      0s
       1    1.5000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.500000000e+00


2024-09-19 14:52:43.429 | INFO     | __main__:<module>:7 - question 25 obtain answer
2024-09-19 14:52:43.431 | INFO     | __main__:<module>:7 - question 26 obtain answer
2024-09-19 14:52:43.433 | INFO     | __main__:<module>:7 - question 27 obtain answer
2024-09-19 14:52:43.435 | INFO     | __main__:<module>:7 - question 28 obtain answer
2024-09-19 14:52:43.436 | INFO     | __main__:<module>:7 - question 29 obtain answer
2024-09-19 14:52:43.440 | INFO     | __main__:<module>:7 - question 30 obtain answer
2024-09-19 14:52:43.442 | INFO     | __main__:<module>:7 - question 31 obtain answer
2024-09-19 14:52:43.445 | INFO     | __main__:<module>:7 - question 32 obtain answer
2024-09-19 14:52:43.449 | INFO     | __main__:<module>:7 - question 33 obtain answer
2024-09-19 14:52:43.452 | INFO     | __main__:<module>:7 - question 34 obtain answer
2024-09-19 14:52:43.453 | INFO     | __main__:<module>:7 - question 35 obtain answer
2024-09-19 14:52:43.455 | INFO     | __main__:<module>:7 - questi

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5700X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 2 columns and 7 nonzeros
Model fingerprint: 0x5eedd26c
Coefficient statistics:
  Matrix range     [3e-01, 8e+02]
  Objective range  [5e+02, 8e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+04]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.6238500e+04   4.834347e+02   0.000000e+00      0s
       1    2.8125000e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.812500000e+04


2024-09-19 14:52:43.496 | INFO     | __main__:<module>:7 - question 57 obtain answer
2024-09-19 14:52:43.497 | INFO     | __main__:<module>:7 - question 58 obtain answer
2024-09-19 14:52:43.499 | INFO     | __main__:<module>:7 - question 59 obtain answer
2024-09-19 14:52:43.500 | INFO     | __main__:<module>:7 - question 60 obtain answer
2024-09-19 14:52:43.502 | INFO     | __main__:<module>:7 - question 61 obtain answer
2024-09-19 14:52:43.503 | INFO     | __main__:<module>:7 - question 62 obtain answer
2024-09-19 14:52:43.505 | INFO     | __main__:<module>:7 - question 63 obtain answer
2024-09-19 14:52:43.506 | INFO     | __main__:<module>:7 - question 64 obtain answer
2024-09-19 14:52:43.509 | INFO     | __main__:<module>:7 - question 65 obtain answer
2024-09-19 14:52:43.511 | INFO     | __main__:<module>:7 - question 66 obtain answer
2024-09-19 14:52:43.512 | INFO     | __main__:<module>:7 - question 67 obtain answer
2024-09-19 14:52:43.514 | INFO     | __main__:<module>:7 - questi

In [20]:
answers = [a if a != 'None' else str(np.inf) for a in answers]

In [21]:
def mark(pred, real, error: float) -> List[bool]:    
    correct = []
    for p, r in zip(pred, real):
        if p == 'Error':
            continue
        if float(r) != 0:
            if (float(p) == np.inf and float(r) == np.inf) or (abs(float(p) - float(r)) / float(r) < error):
                correct.append(True)
            else:
                correct.append(False)
        else:
            if float(p) < error:
                correct.append(True)
            else:
                correct.append(False)
    return correct

In [23]:
error1, error2 = 1e-2, 1e-4
print(f"Under {error1} error rate: the accuracy is {sum(mark(pred_answers, answers, error=error1)) / len(answers)}")
print(f"Under {error2} error rate: the accuracy is {sum(mark(pred_answers, answers, error=error2)) / len(answers)}")
print(f"Under {1e-6} error rate: the accuracy is {sum(mark(pred_answers, answers, error=1e-6)) / len(answers)}")

Under 0.01 error rate: the accuracy is 0.6076388888888888
Under 0.0001 error rate: the accuracy is 0.5902777777777778
Under 1e-06 error rate: the accuracy is 0.5902777777777778
